In [1]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)

In [2]:
from web3 import AsyncWeb3, Web3
from web3.eth import Contract
from datetime import datetime


import json


def read_abi(token: str):
    with open(f"../config/{token.lower()}_abi.json") as f:
        data = json.load(f)
        return data

In [3]:
w3 = Web3(Web3.HTTPProvider("https://bitter-wandering-feather.arbitrum-mainnet.quiknode.pro/862a558ad28be94cf6b1ccae509bdca74a19086a"))

rockonyx_delta_neutral_vault_abi = read_abi("rockonyxdeltaneutralvault")
vault_contract = w3.eth.contract(
    address="0x4a10C31b642866d3A3Df2268cEcD2c5B14600523",
    abi=rockonyx_delta_neutral_vault_abi,
)


In [4]:

def get_shares(vault_contract: Contract, user: str, decimals = 1e6, block_number=200199390):
    tvl = vault_contract.functions.balanceOf(user).call(block_identifier=block_number)

    return tvl / decimals


get_shares(vault_contract, Web3.to_checksum_address("0x9C70305f78a715cb79299a93C5E826310b86865E"), block_number=287646145)

0.0

In [7]:

import pandas as pd

# Read the CSV file
df = pd.read_csv('data/export-advanced-filtered-1734922257222.csv')

# Display the first few rows to verify the data
df.head()


,Transaction Hash,Status,Type,Method,Block,Date Time (UTC),From,Amount,Asset,Shares,pps
0,0xae3adc1494b2320192650d8272b1c2e2d8ce6530bb6a...,Success,ERC-20,Deposit,287585569,12/22/2024 23:49,0x3559B7256E235109f7519b457060ED37106A91FB,1834.620461,USD Coin(USDC),1730.121144,1.0604
1,0x63ab9383b60272e88d9581f6501bb44628bc56c73b2a...,Success,ERC-20,Deposit,287515292,12/22/2024 18:55,0x542654683e236d10d01aA90E285B8018919423A2,"1,021.90",USD Coin(USDC),963.694832,NaN
2,0xdfa05ea0573715e9072472db3e3b2c7dfac1bb6dffca...,Success,ERC-20,Deposit,287514317,12/22/2024 18:51,0x6B50D3b96C7A1E15E05CB133faa1B10a961cbB6a,"1,021.90",USD Coin(USDC),963.694832,NaN
3,0x468a8a8156d371428387fe46248641c17a9a03692a9a...,Success,ERC-20,Deposit,287507184,12/22/2024 18:21,0xb64B5c188b113D8731314B778C4B57d0824A71CB,200,USD Coin(USDC),188.608072,NaN
4,0x8a9be167bfd04f3b0c2050e1a03022d553119a69c45e...,Success,ERC-20,Deposit,287490102,12/22/2024 17:09,0x5168a6b2195A0dB883eA91BDb4B8c36aB8243B90,19.600278,USD Coin(USDC),18.483853,NaN


In [10]:
# Create a new column with shares calculation
df['Prev_Shares'] = df.apply(
    lambda row: get_shares(
        vault_contract,
        Web3.to_checksum_address(row['From']),
        block_number=row['Block'] - 1
    ),
    axis=1
)

# Display the results
df[['From', 'Block', 'Shares', 'Prev_Shares']].head()

,From,Block,Shares,Prev_Shares
0,0x3559B7256E235109f7519b457060ED37106A91FB,287585569,1730.121144,0.0
1,0x542654683e236d10d01aA90E285B8018919423A2,287515292,963.694832,0.0
2,0x6B50D3b96C7A1E15E05CB133faa1B10a961cbB6a,287514317,963.694832,0.0
3,0xb64B5c188b113D8731314B778C4B57d0824A71CB,287507184,188.608072,0.0
4,0x5168a6b2195A0dB883eA91BDb4B8c36aB8243B90,287490102,18.483853,0.0
